In [1]:
%cd ../

/u02/thanhhm/research-idea


In [2]:
import plasma.parallel_processing as pp
import time

/u02/thanhhm/research-idea/.env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def compute(data):
    time.sleep(1)
    return data * 2

def print_pass(data):
    print(f'{data / 2**5}')
    return data

In [4]:
class Outer(pp.communicators.Flow):

    def __init__(self):
        super().__init__()
        manager = pp.mp.Manager()

        self.compute1 = Inner()
        self.compute2 = compute
        self.compute3 = Inner()
        self.printer = print_pass
        self.aggregator = pp.communicators.Aggregator(20)

        self.registerIOs(
            compute1={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            compute2=pp.queues.ProcessQueue(2),
            compute3={
                'compute1': pp.queues.ProcessQueue(2),
                'compute2': pp.queues.ProcessQueue(2),
            },
            printer=pp.queues.ProcessQueue(),
            aggregator=pp.queues.TransferQueue(),
        )


class Inner(pp.communicators.Flow):

    def __init__(self):
        super().__init__()

        self.compute1 = compute
        self.compute2 = compute

caller = Outer()
caller

compute1-<class '__main__.Inner'>
	compute1-<function compute at 0x7f076932c860>
	compute2-<function compute at 0x7f076932c860>
compute2-<function compute at 0x7f076932c860>
compute3-<class '__main__.Inner'>
	compute1-<function compute at 0x7f076932c860>
	compute2-<function compute at 0x7f076932c860>
printer-<function print_pass at 0x7f076917ea20>
aggregator-<class 'plasma.parallel_processing.communicators.aggregator.Aggregator'>

In [5]:
with caller:
    for i in range(20):
        caller.put(i)
    
    print(caller.aggregator.wait())

  0%|          | 0/20 [00:04<?, ?it/s]

0.0
1.0


 10%|█         | 2/20 [00:05<00:00, 19.79it/s]

2.0
3.0


 20%|██        | 4/20 [00:06<00:04,  3.34it/s]

...

16.0
17.0


 90%|█████████ | 18/20 [00:13<00:00,  2.03it/s]

18.0
19.0


100%|██████████| 20/20 [00:14<00:00,  1.43it/s]

[0, 32, 64, 96, 128, 160, 192, 224, 256, 288, 320, 352, 416, 384, 448, 480, 512, 544, 576, 608]
